In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import pprint
import time

In [2]:
# Import city_bikes from part 1
station = pd.read_csv('C:\\Users\\CFS\\Desktop\\Data analysis\\Python\Project_ Statistical-Modelling-with-Python\\Statistical-Modelling-with-Python\\notebooks\\city_bikes.csv')
station.head()

,name,id,empty_slots,free_bikes,latitude,longitude,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.renting,extra.returning,extra.slots,extra.uid
0,10th & Cambie,7a19c49f486d7c0c02b3685d7b240448,21,12,49.262487,-123.114397,2023-09-21 05:01:08.937000+00:00,0,True,1695272339,12,True,True,36,1
1,Yaletown-Roundhouse Station,32603a87cfca71d0f7dfa3513bad69d5,5,11,49.274566,-123.121817,2023-09-21 05:01:08.934000+00:00,2,True,1695272061,9,True,True,16,4
2,Dunsmuir & Beatty,6d42fa40360f9a6b2bf641c7b8bb2862,10,16,49.279764,-123.110154,2023-09-21 05:01:08.934000+00:00,0,True,1695272185,16,True,True,26,5
3,12th & Yukon (City Hall),66f873d641d448bd1572ab086665a458,12,4,49.260599,-123.113504,2023-09-21 05:01:08.936000+00:00,4,True,1695272311,0,True,True,16,7
4,8th & Ash,485d4d24c803cfde829ab89699fed833,14,2,49.264215,-123.117772,2023-09-21 05:01:08.941000+00:00,0,True,1695272244,2,True,True,16,8


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [3]:
# Fetch the value of the environment variable
api_key = os.getenv('Project-2023-LHL')

In [4]:
# Creating funtion
def foursquare(latitude, longitude,station_id, radius, api_key, categories):
        
    # Construct the URL for the Foursquare API
    url = 'https://api.foursquare.com/v3/places/search'
    params = { "ll": f"{latitude},{longitude}", "radius": radius, "categoryId": categories,'fields': 'name,popularity,rating,price,distance,fsq_id'}

    headers = { "Accept": "application/json", "Authorization": api_key }
    
    response = requests.get(url, params=params, headers=headers)
    
    # Initialize an empty list to store the results
    restaurents_bars = []
    
    if response.status_code != 200:
        return f"Error: {response.status_code}"
    else:
        data = response.json()
        for i in data['results']:
            details = {
            'name': i.get('name', None),
            'popularity': i.get('popularity', None),
            'rating': i.get('rating', None),
            'price': i.get('price', None),
            'distance': i.get('distance', None),
            'fsq_id': i.get('fsq_id', None)
        }
            restaurents_bars.append(details)
        return restaurents_bars

all_results = []  # Initialize empty list to accumulate results

# Loop latitude,longitude from station to get details
for index, row in station.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    station_id = row['id']
    result_list = foursquare(latitude, longitude, station_id, 1000, api_key, '4d4b7105d754a06374d81259,4bf58dd8d48988d116941735') # Category ID
    
    for result in result_list:
        # Create a new dictionary with latitude and longitude, id of station, then update it with the result dictionary
        combined_result = {'latitude': latitude, 'longitude': longitude, 'station_id': station_id}
        combined_result.update(result)
        
        # Append the combined dictionary to 'all_results'
        all_results.append(combined_result)
        
# Convert the 'all_results' list of dictionaries to a DataFrame
df_foursquare = pd.DataFrame(all_results)
df_foursquare

,latitude,longitude,station_id,name,popularity,rating,price,distance,fsq_id
0,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Marulilu Cafe,0.978126,8.4,2.0,96,4b1db335f964a520211424e3
1,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,La Taqueria Pinche Taco Shop,0.913264,8.5,2.0,164,4e223f06d4c0d32590f80ff4
2,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Whole Foods,0.996840,8.7,3.0,182,4aa73d25f964a520704c20e3
3,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Elysian Coffee Broadway,0.956118,8.1,1.0,225,4aa98676f964a520105420e3
4,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Menchies Frozen Yogurt,0.918632,8.2,1.0,258,4f0f9e5ee4b0725fea69f095
...,...,...,...,...,...,...,...,...,...
2438,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,Beyond Bread Ltd,0.900051,8.8,1.0,388,4d9bbf48f41ba1430326e8b3
2439,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,Mac Falafel,0.897634,7.0,1.0,247,4aa73ce3f964a5206f4c20e3
2440,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,Yuji's From Japan,0.942220,7.6,2.0,340,51e208e7498ec695529ef159
2441,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,Banyen Books & Sound,0.941402,8.3,NaN,447,4be739cacf200f47c471153c


In [5]:
# Save dataframe to csv
import os

# Define the path
path ='C:\\Users\\CFS\\Desktop\\Data analysis\\Python\\Project_ Statistical-Modelling-with-Python\\Statistical-Modelling-with-Python\\notebooks'

# Creating the full file path
file_path = os.path.join(path, 'df_foursquare.csv')

# Save the DataFrame
df_foursquare.to_csv(file_path, index=False)

# Yelp

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [6]:
# Fetch the value of the environment variable
api_key1 = os.getenv('xcCiqFdgljmjPUDbS5V40w')

In [7]:
# Creating funtion
def yelp(latitude, longitude, station_id,radius, api_key1, categories):
        
    # Construct the URL for the Yelp API
    url = 'https://api.yelp.com/v3/businesses/search'
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'categories': categories,
    }
    headers = {
        "Authorization": 'bearer ' + api_key1
    }
    response = requests.get(url, params=params, headers=headers)
    
    # Initialize an empty list to store the results
    restaurents_bars = []

    if response.status_code == 429:
        print("Rate limit reached. Sleeping for 60 seconds...")
        time.sleep(60)
        return "RateLimitError"
    
    if response.status_code != 200:
        return f"Error: {response.status_code}"
    else:
        data = response.json()
        for i in data['businesses']:
            details = {
                'name': i.get('name', None),
                'rating': i.get('rating', None),
                'price': i.get('price', None),
                'distance': i.get('distance', None),
                'review_count' : i.get('review_count', None),
                'yelp_id':i.get('id', None),
            }
            restaurents_bars.append(details)
        return restaurents_bars

all_results1 = []  # Initialize empty list to accumulate results

# Loop latitude,longitude from station to get details
for index, row in station.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    station_id = row['id']
    result_list1 = yelp(latitude, longitude, station_id, 1000, api_key1, 'bars,restaurants') # Category ID

    if result_list1 == "RateLimitError":
        print("Skipping this iteration due to rate limit.")
        continue
    elif not isinstance(result_list1, list):
        print(f"Skipping this iteration due to an error: {result_list1}")
        continue

    for result1 in result_list1:
        combined_result1 = {'latitude': latitude, 'longitude': longitude, 'station_id': station_id}
        combined_result1.update(result1)
        all_results1.append(combined_result1)

df_yelp = pd.DataFrame(all_results1)
df_yelp

,latitude,longitude,station_id,name,rating,price,distance,review_count,yelp_id
0,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,La Taqueria Pinche Taco Shop,4.0,$$,169.517456,681,6iOAgzJ0DRZNSKA3FSrrOg
1,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Saku,4.5,$$,178.845344,226,XAH2HpuUUtu7CUO26pbs4w
2,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Marulilu Cafe,4.0,$$,102.329817,286,NensKn1MSVU_rm-1Y6WlFA
3,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,iDen & Quan Ju De Beijing Duck House,3.5,None,263.641336,106,Zi8Ywk36Ws_4zjw2gjMMFA
4,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Uma Sushi,4.5,$$$,152.157897,114,4118Aq9LbkvUr4s719uUkA
...,...,...,...,...,...,...,...,...,...
4859,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,Green Leaf Sushi,4.0,$$,569.394966,397,irdrKokOvuxP_atEjtzkQA
4860,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,Double DD Pizza,3.5,$$,521.927806,87,AejLer68UaDsVrP4xXZ_kw
4861,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,Aphrodite's Organic Pie Shop,3.5,$$,456.914922,259,mROKrBThkXlv_Xo__OTOmQ
4862,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,La Notte Restaurant,4.0,$$,929.874867,56,H2tZljTwEyTPHrFLCGkxcg


In [11]:
# Change type of 'price' in df_yelp dataframe
def change_price(x):
    if x == '$':
        return '1'
    elif x == '$$':
        return '2'
    elif x == '$$$':
        return '3'
    elif x == '$$$$':
        return '4'
df_yelp['price'] = df_yelp['price'].apply(change_price)
df_yelp

,latitude,longitude,station_id,name,rating,price,distance,review_count,yelp_id
0,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,La Taqueria Pinche Taco Shop,4.0,None,169.517456,681,6iOAgzJ0DRZNSKA3FSrrOg
1,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Saku,4.5,None,178.845344,226,XAH2HpuUUtu7CUO26pbs4w
2,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Marulilu Cafe,4.0,None,102.329817,286,NensKn1MSVU_rm-1Y6WlFA
3,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,iDen & Quan Ju De Beijing Duck House,3.5,None,263.641336,106,Zi8Ywk36Ws_4zjw2gjMMFA
4,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Uma Sushi,4.5,None,152.157897,114,4118Aq9LbkvUr4s719uUkA
...,...,...,...,...,...,...,...,...,...
4859,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,Green Leaf Sushi,4.0,None,569.394966,397,irdrKokOvuxP_atEjtzkQA
4860,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,Double DD Pizza,3.5,None,521.927806,87,AejLer68UaDsVrP4xXZ_kw
4861,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,Aphrodite's Organic Pie Shop,3.5,None,456.914922,259,mROKrBThkXlv_Xo__OTOmQ
4862,49.265442,-123.187738,cc25ae4f093b33ba0afd1dbc0dd20324,La Notte Restaurant,4.0,None,929.874867,56,H2tZljTwEyTPHrFLCGkxcg


In [10]:
# Save dataframe to csv
import os

# Define the path
path ='C:\\Users\\CFS\\Desktop\\Data analysis\\Python\\Project_ Statistical-Modelling-with-Python\\Statistical-Modelling-with-Python\\notebooks'

# Creating the full file path
file_path = os.path.join(path, 'df_yelp.csv')

# Save the DataFrame
df_yelp.to_csv(file_path, index=False)

# Comparing Results

After executing the queries below, I got the result:
  
- API Yelp provided more loactions than API Foursquare because: Number of unique IDs of bars & restaurants at Foursquare: 593 locations. Number of unique IDs of bars & restaurants at Yelp: 827 location.

 - Based on df foursquare.info() df_yelp.info(), we can see Yelp gives more details than Foursquare at price and rating

In [ ]:
# Check column missing
cols_missing_vals = ['rating', 'price','distance','popularity']
(df_foursquare[cols_missing_vals] == 0).sum()

rating        0
price         0
distance      0
popularity    0
dtype: int64

In [ ]:
# Check column missing
cols_missing_vals = ['rating', 'price','distance','review_count']
(df_yelp[cols_missing_vals] == 0).sum()

rating          0
price           0
distance        0
review_count    0
dtype: int64

In [ ]:
df_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4864 entries, 0 to 4863
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   latitude      4864 non-null   float64
 1   longitude     4864 non-null   float64
 2   station_id    4864 non-null   object 
 3   name          4864 non-null   object 
 4   rating        4864 non-null   float64
 5   price         4329 non-null   float64
 6   distance      4864 non-null   float64
 7   review_count  4864 non-null   int64  
 8   yelp_id       4864 non-null   object 
dtypes: float64(5), int64(1), object(3)
memory usage: 342.1+ KB


In [ ]:
df_foursquare.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2443 entries, 0 to 2442
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    2443 non-null   float64
 1   longitude   2443 non-null   float64
 2   station_id  2443 non-null   object 
 3   name        2443 non-null   object 
 4   popularity  2443 non-null   float64
 5   rating      2406 non-null   float64
 6   price       1689 non-null   float64
 7   distance    2443 non-null   int64  
 8   fsq_id      2443 non-null   object 
dtypes: float64(5), int64(1), object(3)
memory usage: 171.9+ KB


In [ ]:
# Check number unique ids of bars & restaurant at Foursquare and Yelp
id_foursquare = df_foursquare['fsq_id'].nunique()
id_yelp = df_yelp['yelp_id'].nunique()

print(f"Number of unique IDs of bars & restaurants at Foursquare: {id_foursquare} locations.")
print(f"Number of unique IDs of bars & restaurants at Yelp: {id_yelp} locations.")

Number of unique IDs of bars & restaurants at Foursquare: 593 locations
Number of unique IDs of bars & restaurants at Yelp: 827 locations


Get the top 10 restaurants according to their rating

In [ ]:
df_foursquare.sort_values('rating', ascending = False).drop_duplicates('name').head(10)

,latitude,longitude,station_id,name,popularity,rating,price,distance,fsq_id
488,49.291909,-123.140713,00fa94ad698dc4a9e4d708d6fd32f294,Stanley Park English Bay Seawall,0.999778,9.6,NaN,483,4ba69381f964a520ad5f39e3
1052,49.276294,-123.106534,1648ead5ea617d2cea115869d9516f5e,Seaside Seawall,0.999013,9.5,NaN,453,4df29cb0fa76abc3d86a240a
1663,49.285512,-123.138400,d085c6749a32517e1fb0d070e4128402,English Bay Beach,0.999562,9.5,NaN,344,4bcbdf5268f976b049e86183
193,49.289124,-123.121960,664502c28e8147615f17b8ea355f40d0,Coal Harbour Seawall,0.994003,9.5,NaN,260,4ab03489f964a520706620e3
2308,49.249184,-123.116141,549f00f6039e8bb702a00dcca08a8ed0,Queen Elizabeth Park,0.999772,9.4,NaN,870,4bdf6cddffdec9287a09eca1
2203,49.274232,-123.129998,6620c54bd6b3a79800b2b67e999b24d0,False Creek Seawall,0.998540,9.4,NaN,322,4c5385244623be9aa29e36f1
1716,49.266476,-123.109389,7ed043a19e6a4ece742fe747b663bdbf,33 Acres Brewing Co,0.950404,9.4,2.0,413,51b201d27dd249ae714ba728
345,49.272827,-123.147744,b9fd847d53fe45658fee4c46e7f562b8,Kitsilano Beach,0.999948,9.4,NaN,503,4aa7e39df964a520dc4d20e3
1480,49.302577,-123.155616,45a4c865db23e1cad0209afa5a799de0,Third Beach,0.999480,9.4,NaN,103,4aaafb5ef964a5205e5820e3
222,49.273663,-123.127075,3a4131fa94946d93d3529136efd62413,David Lam Park,0.998972,9.3,NaN,268,4b572f20f964a520f92928e3


In [ ]:
df_yelp.sort_values('rating',ascending= False).drop_duplicates('name').head(10)

,latitude,longitude,station_id,name,rating,price,distance,review_count,yelp_id
3372,49.283298,-123.142288,8e93d0da1ee6caadf21ed9575b5fa9da,Number E Food,5.0,2.0,1036.384142,143,kDZTaeUaf78Jd25ktX8a8g
1236,49.267222,-123.153157,826af7eee4ce28b6fa4b4f0211305792,NUTTEA,5.0,NaN,295.447339,28,OXEaIQsgJIUCyRev5VgvLA
746,49.262598,-123.138921,96bf713e60bc35cc2feb5fd53a4f3471,Mazahr Lebanese Kitchen,5.0,NaN,130.531259,51,ForB55b3mE5EEhC25sHMnQ
3773,49.263523,-123.155267,2681455ba2d97283998a806b14d3e95c,Livni Café & Bakery,5.0,NaN,64.821245,7,tUh4NuINdnkbVF16iPc2wQ
4283,49.247854,-123.153422,085a0e120a08a80a58c0ae0ff9da565a,Trim & Healthy,5.0,NaN,591.163894,1,D-J3i1WCZxcdjixVSTrHCQ
2899,49.283236,-123.070443,c52e51e3288a00993bdf892fdc2c2c2c,Elephant,5.0,NaN,366.341060,10,y4vjMBAda7zdSglUnurecg
650,49.281506,-123.120101,6ddeafa873279951630a00191333c45d,Incognito Coffee,5.0,2.0,130.377049,180,3ByjrLF8nyArUxvO6vqJCw
2537,49.270684,-123.073562,f387ec226c432c5c054e3d3616d90501,Viet Family- The Vegan House,5.0,NaN,304.344034,18,je53sE7RTFj9Nix0aatH3g
378,49.288444,-123.139203,1d7e73b3321ebb6713774d376247d9f9,Arike Restaurant,5.0,NaN,221.084330,25,Q7d2jLllssfrP9u3JJGMIw
4668,49.277178,-123.130000,cc2084e365f6f230fe26bae0b6867775,Manoush'eh,5.0,1.0,316.936126,235,K1nbiOrySlw_-XG-3NmErQ
